In [40]:
from fastapi import FastAPI, Query, HTTPException
from sqlalchemy import create_engine, text
import pandas as pd

app = FastAPI()

# connect to the database created in question2
database_uri = "sqlite:///../data/my_database.db"
engine = create_engine(database_uri, pool_pre_ping=True)

@app.get("/average_price") #execute following function when get request made to /average_price endpoint
async def get_average_price(metal: str = Query(..., description="Metal name (e.g., 'Copper', 'Zinc')"),
                            start_date: str = Query(..., description="Start date (YYYY-MM-DD)"),
                            end_date: str = Query(..., description="End date (YYYY-MM-DD)")):
    
    # ensure that start_date is before end_date
    if datetime.strptime(start_date, '%Y-%m-%d').date() > datetime.strptime(end_date, '%Y-%m-%d').date():
        raise HTTPException(status_code=400, detail="Invalid date range. Start date must be before end date.")
    
    # query the database to calculate the average price for the specified metal and date range
    query = text(f"""
        SELECT AVG(Price) AS avg_price
        FROM MetalPrices
        WHERE Metal = :metal
        AND Date >= :start_date
        AND Date <= :end_date
    """)
    
    with engine.connect() as conn:
        result = conn.execute(query, metal=metal, start_date=start_date, end_date=end_date)
        avg_price = result.scalar()
    
    if avg_price is None:
        raise HTTPException(status_code=404, detail=f"No data found for {metal} between {start_date} and {end_date}")
    
    return {"metal": metal, "start_date": start_date, "end_date": end_date, "average_price": avg_price}

In [33]:
!jupyter nbconvert --to script Question4.ipynb
#convert the jupyter notebook to a python script so that we can run the app

[NbConvertApp] Converting notebook Question4.ipynb to script
[NbConvertApp] Writing 2566 bytes to Question4.py


In [42]:
import requests

#define the URL of your FastAPI app endpoint
url = "http://127.0.0.1:8000/average_price"

#define query parameters for the request
params1 = {
    "metal": "Copper",
    "start_date": "2020-01-01",
    "end_date": "2021-12-31"
}

params2 = {
    "metal": "Zinc_RSI",
    "start_date": "2021-09-01",
    "end_date": "2021-12-31"
}


params3 = {
    "metal": "Copper_MACD",
    "start_date": "2022-09-01",
    "end_date": "2021-12-31"
}

# Send the get request
response1 = requests.get(url, params=params1)

response2 = requests.get(url, params=params2)
#this one should not work
response3 = requests.get(url, params=params3)

In [43]:
response1.json()

{'metal': 'Copper',
 'start_date': '2020-01-01',
 'end_date': '2021-12-31',
 'average_price': 7736.883141762452}

In [44]:
response2.json()

{'metal': 'Zinc_RSI',
 'start_date': '2021-09-01',
 'end_date': '2021-12-31',
 'average_price': 54.913400548983816}

In [45]:
response3.json()

{'detail': 'Invalid date range. Start date must be before end date.'}